In [1]:
from sqlalchemy import create_engine
import pandas as pd

DB_USER = "callact_admin"
DB_PASSWORD = "callact_pwd1"
DB_HOST = "localhost"
DB_PORT = 5433
DB_NAME = "callact_db"

engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# 총 건수
pd.read_sql("select count(*) as cnt from consultations", engine)


,cnt
0,6533


In [2]:
# # 예시 20건
# pd.read_sql("""
#     select
#         id, customer_id, agent_id, status, category, title,
#         call_date, call_time, call_duration, fcr, is_best_practice,
#         quality_score, created_at
#     from consultations
#     order by created_at desc nulls last
#     limit 20
# """, engine)


In [3]:
# df_consults = pd.read_sql("""
#     select id, customer_id, agent_id, status, category, title, created_at
#     from consultations
#     order by created_at desc nulls last
#     limit 10
# """, engine)

# df_consults


In [4]:
# sample_id = df_consults.loc[0, "id"]

# df_docs = pd.read_sql("""
#     select id, consultation_id, document_type, title, content
#     from consultation_documents
#     where consultation_id = %(cid)s
#     limit 3
# """, engine, params={"cid": sample_id})

# df_docs


In [5]:
# print(df_docs.loc[0, "content"])


In [6]:
# pd.set_option("display.max_colwidth", None)
# df_docs[["title","content"]]


In [7]:
# df_emp = pd.read_sql("""
#     select
#         id, name, email, role, department, status,
#         consultations, fcr, "avgTime", rank, created_at
#     from employees
#     order by created_at desc nulls last
#     limit 20
# """, engine)

# df_emp


In [8]:
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", 200)
# df_emp[["id","name","department","consultations","fcr","avgTime","rank"]]


In [9]:
# pd.read_sql("""
#   select column_name, data_type, udt_name
#   from information_schema.columns
#   where table_name = 'consultation_documents'
#   order by ordinal_position
# """, engine)


## 테스트 셀

In [10]:
from pathlib import Path
import sys

# Ensure project root is on sys.path so `import app.*` works
_root = Path.cwd().resolve()
while _root != _root.parent and not (_root / "app").exists():
    _root = _root.parent
if str(_root) not in sys.path:
    sys.path.insert(0, str(_root))

import os

def load_env(path):
    if path.exists():
        for line in path.read_text().splitlines():
            line = line.strip()
            if not line or line.startswith("#") or "=" not in line:
                continue
            k, v = line.split("=", 1)
            os.environ.setdefault(k.strip(), v.strip().strip('"').strip("'"))

load_env(Path(".env"))
load_env(Path(".env.local"))

# 필요시 직접 override
os.environ.setdefault("DB_HOST", "localhost")
os.environ.setdefault("DB_PORT", "5433")


'5433'

In [11]:
import asyncio
from app.rag.pipeline.pipeline import run_rag, route

query = "다둥이 카드 혜택이 어떻게 되나요?"
routing = route(query)
routing


{'route': 'card_info',
 'filters': {},
 'ui_route': 'card_info',
 'db_route': 'card_tbl',
 'boost': {},
 'query_template': None,
 'matched': {'card_names': [],
  'actions': [],
  'payments': [],
  'weak_intents': ['혜택']},
 'applepay_intent': None,
 'should_search': True,
 'should_trigger': True,
 'should_route': True,
 'document_sources': ['guide_merged', 'guide_general'],
 'exclude_sources': ['terms'],
 'document_source_policy': 'A',
 'need_consult_case_search': True,
 'consult_category_candidates': ['분실도난', '혜택'],
 'consult_keyword_hits': 0}

In [12]:
# Jupyter/IPython already runs an event loop; use await
res = await run_rag(query)
res.keys()


[retriever] table=card_products fetch_ms=0.0 cand_added=2 total_cand=2 break_hit=False
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=124.9 rows=1
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=123.8 rows=1
[retriever] source=guide_all fetch_ms=0.0 cand_added=0 total_cand=2 break_hit=False
[pipeline_retrieve] retrieve_ms=383.7 doc_count=3 route=card_info
[cards] llm_input_chars=1311 ctx_chars=675 doc_ids=['dadungi_013', 'CARD-SHINHAN-신한카드-Point-Plan-(서울시다둥이행복카드)', 'CARD-SHINHAN-신한카드-Point-Plan-체크-(서울시다둥이행복카드)'] doc_chars=[110, 283, 282]
[card_cache] set layer=redis key=02c53310b7363bd3 ttl=120
[card_cache] set layer=mem key=02c53310b7363bd3 ttl=120
[rag] route=1.6ms retrieve=1182.3ms cards=2264.0ms post=0.1ms total=3448.0ms docs=3 route=card_info cache=miss retrieve_cache=miss


dict_keys(['currentSituation', 'nextStep', 'guidanceScript', 'guide_script', 'routing', 'meta', 'docs', 'consult_docs'])

In [13]:
for i, doc in enumerate(res.get("consult_docs", [])[:3], 1):
    print(f"\n[{i}] {doc.get('title')}")
    print(doc.get("content", "")[:500])



[1] 이용방법 안내 상담
상담사: 상담원 [상담원명#1]입니다.
손님: 예, 제가 카드 실적을 충족을 하면 이거를 어떻게 혜택을 보는 건가요?
상담사: 네, 해당 부분 확인 도와드릴 텐데요, 먼저 [고객명#1]님 본인 맞으십니까?
손님: 네.
상담사: 네, 확인 감사합니다. 지금 손님께서 [카드사명#1]카드 이용 중이신 걸로 확인되시는데 맞으실까요?
손님: 네.
상담사: 예.
상담사: 손님께서 이 카드로 그 [금액#1] 구매하신 적 있으시죠?
손님: 네.
상담사: 음 그러면은 이제 [금액#2]를 할부로 구매하신 걸로 확인되시는데요. 지난달 실적 [금액#2] 이상 충족해 주시면은 [금액#2] 할부 금액이 매월 나오잖아요? 그 금액에서 60만 원 이상일 경우에 2만 원 30만 원 이상일 경우 만 원 할부 건에 대해서 할인해드리고 청구해드리고 있습니다.
손님: 네.
손님: 그러면 청구서를 조회를 하면 이게 어떻게 나타나나요?
상담사: 지금 보시면은 이제 취소 매출 반영 금액으로 확인되실까요?
손님: 잠시만요. 그

[2] 이벤트 안내 상담
상담사: 상담원 [상담원명#1]입니다. 
손님: 아 네. 저 그 통신비 때문에 엘지 유플러스에서 [카드사명#1] 만들려고 하는데
상담사: 네, 그러세요?
손님: 네, 왜 프로모션 대상이 아닌 거죠?
상담사: 어 대상이 아님을 확인하셨어요, 손님?
손님: 네.
상담사: 어 그러셨어요? 혹시 최근 6개월에 저희 [카드사명#1] 결제하셨던 이력이 있으셨던 건가요?
손님: 아니, 없는데.
상담사: 그러셨어요? 그런데 그 대상이 아닌 건 어떻게 확인하셨습니까? 
손님: 그, 저기가, 저기 뭐야. 
손님: 그 통신사 직원이 보내주셔가지고.
상담사: 네.
손님: 이벤트 대상인지 확인해 보니까 아니라고 하는데요. 그 링크 보내주셔가지고 했더니.
상담사: 링크로 하셨다고요?
손님: 네.
상담사: 기존에 그러면은 손님 한번 저희가 심사를 진행해 보셨던 거예요, 손님?
손님: 일단 그거는 해봤거든요. 거기는 뭐 비밀번호랑 제 핸드폰 번호를 쓰기는

In [14]:
len(res.get("docs", [])), res.get("docs", [])[:3]


(3,
 [{'id': 'dadungi_013',
   'db_id': 'dadungi_013',
   'title': '서울시다둥이행복카드필수 생활비 적립 서비스 주말 배달앱 포인트 적립 - 배달앱 건당 2만원 이상 결제 시 1천 포인트 적립 편의점 포인트 적립',
   'content': '- CU, GS25, 세븐일레븐 5% 포인트 적립주말 배달앱 포인트 적립 - 배달앱 건당 2만원 이상 결제 시 1천 포인트 적립 편의점 포인트 적립 - CU, GS25, 세븐일레븐 5% 포인트 적립',
   'metadata': {'title': '서울시다둥이행복카드필수 생활비 적립 서비스 주말 배달앱 포인트 적립 - 배달앱 건당 2만원 이상 결제 시 1천 포인트 적립 편의점 포인트 적립',
    'category': '혜택/할인',
    'category1': 'terms',
    'original_id': 'dadungi_013',
    'source_table': 'service_guide_documents',
    'original_source': 'special_card',
    'original_category': '혜택/할인',
    'original_card_name': '서울시다둥이행복카드',
    'preprocessing_date': '2026-01-11',
    'preprocessing_version': '1.0'},
   'table': 'service_guide_documents'},
  {'id': 'CARD-SHINHAN-신한카드-Point-Plan-(서울시다둥이행복카드)',
   'db_id': 'CARD-SHINHAN-신한카드-Point-Plan-(서울시다둥이행복카드)',
   'title': None,
   'content': '테디카드 Point Plan (서울시다둥이행복카드)\n\n\n고객센터\n\n고객센터: 1544-7000\n\n단기/장기 카드대출: 1544-0303\n\n테디카드 Point Plan 카드신청: 

In [15]:
routing = route(query)
routing


{'route': 'card_info',
 'filters': {},
 'ui_route': 'card_info',
 'db_route': 'card_tbl',
 'boost': {},
 'query_template': None,
 'matched': {'card_names': [],
  'actions': [],
  'payments': [],
  'weak_intents': ['혜택']},
 'applepay_intent': None,
 'should_search': True,
 'should_trigger': True,
 'should_route': True,
 'document_sources': ['guide_merged', 'guide_general'],
 'exclude_sources': ['terms'],
 'document_source_policy': 'A',
 'need_consult_case_search': True,
 'consult_category_candidates': ['분실도난', '혜택'],
 'consult_keyword_hits': 0}

In [16]:
res = await run_rag(query, session_state={})
res.keys(), len(res.get("consult_docs", [])), res.get("consult_docs", [])[:1]


[card_cache] hit=1 layer=redis key=02c53310b7363bd3
[rag] route=1.7ms retrieve=510.1ms cards=1.2ms post=0.1ms total=513.0ms docs=3 route=card_info cache=hit(redis) retrieve_cache=hit(redis)


(dict_keys(['currentSituation', 'nextStep', 'guidanceScript', 'guide_script', 'routing', 'meta', 'docs', 'consult_docs']),
 4,
 [{'id': 'hana_consultation_23419',
   'db_id': 'hana_consultation_23419',
   'title': '이용방법 안내 상담',
   'content': '상담사: 상담원 [상담원명#1]입니다.\n손님: 예, 제가 카드 실적을 충족을 하면 이거를 어떻게 혜택을 보는 건가요?\n상담사: 네, 해당 부분 확인 도와드릴 텐데요, 먼저 [고객명#1]님 본인 맞으십니까?\n손님: 네.\n상담사: 네, 확인 감사합니다. 지금 손님께서 [카드사명#1]카드 이용 중이신 걸로 확인되시는데 맞으실까요?\n손님: 네.\n상담사: 예.\n상담사: 손님께서 이 카드로 그 [금액#1] 구매하신 적 있으시죠?\n손님: 네.\n상담사: 음 그러면은 이제 [금액#2]를 할부로 구매하신 걸로 확인되시는데요. 지난달 실적 [금액#2] 이상 충족해 주시면은 [금액#2] 할부 금액이 매월 나오잖아요? 그 금액에서 60만 원 이상일 경우에 2만 원 30만 원 이상일 경우 만 원 할부 건에 대해서 할인해드리고 청구해드리고 있습니다.\n손님: 네.\n손님: 그러면 청구서를 조회를 하면 이게 어떻게 나타나나요?\n상담사: 지금 보시면은 이제 취소 매출 반영 금액으로 확인되실까요?\n손님: 잠시만요. 그 앱에 들어가면 되죠?\n상담사: 네, 어플에서 보셔도 되시고요. 밑에 어플 보시면은 이제 밑에 보시면 혜택이라고 있어요.\n손님: 잠시만요.\n상담사: 혜택은 누르시면요.\n손님: 네, 눌렀어요.\n상담사: 네, 로그인 하기 보이시죠?\n손님: 예, 로그인 됐는데요.\n상담사: 로그인해 주시고 저도 휴대폰 보면서 말씀 드리는거라 조금만 기다려주세요.\n손님: 네.\n상담사: 네, 밑에 보시면은 [날짜#1]월 받은 혜택 실적 조회라고

In [17]:
res = await run_rag(query, session_state={})
len(res.get("consult_docs", [])), res.get("consult_docs", [])[:1]


[card_cache] hit=1 layer=redis key=02c53310b7363bd3
[rag] route=1.6ms retrieve=447.0ms cards=1.2ms post=0.1ms total=449.8ms docs=3 route=card_info cache=hit(redis) retrieve_cache=hit(redis)


(4,
 [{'id': 'hana_consultation_23419',
   'db_id': 'hana_consultation_23419',
   'title': '이용방법 안내 상담',
   'content': '상담사: 상담원 [상담원명#1]입니다.\n손님: 예, 제가 카드 실적을 충족을 하면 이거를 어떻게 혜택을 보는 건가요?\n상담사: 네, 해당 부분 확인 도와드릴 텐데요, 먼저 [고객명#1]님 본인 맞으십니까?\n손님: 네.\n상담사: 네, 확인 감사합니다. 지금 손님께서 [카드사명#1]카드 이용 중이신 걸로 확인되시는데 맞으실까요?\n손님: 네.\n상담사: 예.\n상담사: 손님께서 이 카드로 그 [금액#1] 구매하신 적 있으시죠?\n손님: 네.\n상담사: 음 그러면은 이제 [금액#2]를 할부로 구매하신 걸로 확인되시는데요. 지난달 실적 [금액#2] 이상 충족해 주시면은 [금액#2] 할부 금액이 매월 나오잖아요? 그 금액에서 60만 원 이상일 경우에 2만 원 30만 원 이상일 경우 만 원 할부 건에 대해서 할인해드리고 청구해드리고 있습니다.\n손님: 네.\n손님: 그러면 청구서를 조회를 하면 이게 어떻게 나타나나요?\n상담사: 지금 보시면은 이제 취소 매출 반영 금액으로 확인되실까요?\n손님: 잠시만요. 그 앱에 들어가면 되죠?\n상담사: 네, 어플에서 보셔도 되시고요. 밑에 어플 보시면은 이제 밑에 보시면 혜택이라고 있어요.\n손님: 잠시만요.\n상담사: 혜택은 누르시면요.\n손님: 네, 눌렀어요.\n상담사: 네, 로그인 하기 보이시죠?\n손님: 예, 로그인 됐는데요.\n상담사: 로그인해 주시고 저도 휴대폰 보면서 말씀 드리는거라 조금만 기다려주세요.\n손님: 네.\n상담사: 네, 밑에 보시면은 [날짜#1]월 받은 혜택 실적 조회라고 보이실까요?\n손님: 예.\n손님: 밑으로 내려가면요?\n상담사: 아니요. 메뉴를 올려 보시면 밑에 보이실 거예요. 그 결제창 혜택 위에 보시면은 그거 누르시면요, 지난달이랑 이번 달 실적이랑 인제 받은 혜택까지 확인이

In [18]:
import os
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(
    f"postgresql+psycopg2://{os.environ['DB_USER']}:{os.environ['DB_PASSWORD']}@"
    f"{os.environ['DB_HOST']}:{os.environ['DB_PORT']}/{os.environ['DB_NAME']}"
)


In [19]:
pd.read_sql("""
  select count(*) as total,
         sum(case when embedding is null then 1 else 0 end) as embedding_null
  from consultation_documents
""", engine)


,total,embedding_null
0,6533,0


In [20]:
pd.read_sql("""
  select id, title, category, length(content) as content_len
  from consultation_documents
  order by created_at desc nulls last
  limit 5
""", engine)


,id,title,category,content_len
0,hana_consultation_200560,승인취소/매출취소 안내 상담,승인취소/매출취소 안내,844
1,hana_consultation_200561,안심클릭/[서비스명#1]페이/기타페이 안내 상담,안심클릭/[서비스명#1]페이/기타페이 안내,818
2,hana_consultation_200573,이용방법 안내 상담,이용방법 안내,1695
3,hana_consultation_200559,도난/분실 신청/해제 상담,도난/분실 신청/해제,1662
4,hana_consultation_200562,결제계좌 안내/변경 상담,결제계좌 안내/변경,822


In [21]:
res = await run_rag(query, session_state={})
len(res.get("consult_docs", [])), res.get("consult_docs", [])[:1]


[card_cache] hit=1 layer=redis key=02c53310b7363bd3
[rag] route=2.5ms retrieve=323.0ms cards=0.7ms post=0.1ms total=326.3ms docs=3 route=card_info cache=hit(redis) retrieve_cache=hit(redis)


(4,
 [{'id': 'hana_consultation_23419',
   'db_id': 'hana_consultation_23419',
   'title': '이용방법 안내 상담',
   'content': '상담사: 상담원 [상담원명#1]입니다.\n손님: 예, 제가 카드 실적을 충족을 하면 이거를 어떻게 혜택을 보는 건가요?\n상담사: 네, 해당 부분 확인 도와드릴 텐데요, 먼저 [고객명#1]님 본인 맞으십니까?\n손님: 네.\n상담사: 네, 확인 감사합니다. 지금 손님께서 [카드사명#1]카드 이용 중이신 걸로 확인되시는데 맞으실까요?\n손님: 네.\n상담사: 예.\n상담사: 손님께서 이 카드로 그 [금액#1] 구매하신 적 있으시죠?\n손님: 네.\n상담사: 음 그러면은 이제 [금액#2]를 할부로 구매하신 걸로 확인되시는데요. 지난달 실적 [금액#2] 이상 충족해 주시면은 [금액#2] 할부 금액이 매월 나오잖아요? 그 금액에서 60만 원 이상일 경우에 2만 원 30만 원 이상일 경우 만 원 할부 건에 대해서 할인해드리고 청구해드리고 있습니다.\n손님: 네.\n손님: 그러면 청구서를 조회를 하면 이게 어떻게 나타나나요?\n상담사: 지금 보시면은 이제 취소 매출 반영 금액으로 확인되실까요?\n손님: 잠시만요. 그 앱에 들어가면 되죠?\n상담사: 네, 어플에서 보셔도 되시고요. 밑에 어플 보시면은 이제 밑에 보시면 혜택이라고 있어요.\n손님: 잠시만요.\n상담사: 혜택은 누르시면요.\n손님: 네, 눌렀어요.\n상담사: 네, 로그인 하기 보이시죠?\n손님: 예, 로그인 됐는데요.\n상담사: 로그인해 주시고 저도 휴대폰 보면서 말씀 드리는거라 조금만 기다려주세요.\n손님: 네.\n상담사: 네, 밑에 보시면은 [날짜#1]월 받은 혜택 실적 조회라고 보이실까요?\n손님: 예.\n손님: 밑으로 내려가면요?\n상담사: 아니요. 메뉴를 올려 보시면 밑에 보이실 거예요. 그 결제창 혜택 위에 보시면은 그거 누르시면요, 지난달이랑 이번 달 실적이랑 인제 받은 혜택까지 확인이

In [22]:
route(query)


{'route': 'card_info',
 'filters': {},
 'ui_route': 'card_info',
 'db_route': 'card_tbl',
 'boost': {},
 'query_template': None,
 'matched': {'card_names': [],
  'actions': [],
  'payments': [],
  'weak_intents': ['혜택']},
 'applepay_intent': None,
 'should_search': True,
 'should_trigger': True,
 'should_route': True,
 'document_sources': ['guide_merged', 'guide_general'],
 'exclude_sources': ['terms'],
 'document_source_policy': 'A',
 'need_consult_case_search': True,
 'consult_category_candidates': ['분실도난', '혜택'],
 'consult_keyword_hits': 0}

In [23]:
res = await run_rag(query, session_state={})
len(res.get("consult_docs", [])), res.get("consult_docs", [])[:1]


[card_cache] hit=1 layer=redis key=02c53310b7363bd3
[rag] route=1.6ms retrieve=353.7ms cards=1.5ms post=0.2ms total=357.0ms docs=3 route=card_info cache=hit(redis) retrieve_cache=hit(redis)


(4,
 [{'id': 'hana_consultation_23419',
   'db_id': 'hana_consultation_23419',
   'title': '이용방법 안내 상담',
   'content': '상담사: 상담원 [상담원명#1]입니다.\n손님: 예, 제가 카드 실적을 충족을 하면 이거를 어떻게 혜택을 보는 건가요?\n상담사: 네, 해당 부분 확인 도와드릴 텐데요, 먼저 [고객명#1]님 본인 맞으십니까?\n손님: 네.\n상담사: 네, 확인 감사합니다. 지금 손님께서 [카드사명#1]카드 이용 중이신 걸로 확인되시는데 맞으실까요?\n손님: 네.\n상담사: 예.\n상담사: 손님께서 이 카드로 그 [금액#1] 구매하신 적 있으시죠?\n손님: 네.\n상담사: 음 그러면은 이제 [금액#2]를 할부로 구매하신 걸로 확인되시는데요. 지난달 실적 [금액#2] 이상 충족해 주시면은 [금액#2] 할부 금액이 매월 나오잖아요? 그 금액에서 60만 원 이상일 경우에 2만 원 30만 원 이상일 경우 만 원 할부 건에 대해서 할인해드리고 청구해드리고 있습니다.\n손님: 네.\n손님: 그러면 청구서를 조회를 하면 이게 어떻게 나타나나요?\n상담사: 지금 보시면은 이제 취소 매출 반영 금액으로 확인되실까요?\n손님: 잠시만요. 그 앱에 들어가면 되죠?\n상담사: 네, 어플에서 보셔도 되시고요. 밑에 어플 보시면은 이제 밑에 보시면 혜택이라고 있어요.\n손님: 잠시만요.\n상담사: 혜택은 누르시면요.\n손님: 네, 눌렀어요.\n상담사: 네, 로그인 하기 보이시죠?\n손님: 예, 로그인 됐는데요.\n상담사: 로그인해 주시고 저도 휴대폰 보면서 말씀 드리는거라 조금만 기다려주세요.\n손님: 네.\n상담사: 네, 밑에 보시면은 [날짜#1]월 받은 혜택 실적 조회라고 보이실까요?\n손님: 예.\n손님: 밑으로 내려가면요?\n상담사: 아니요. 메뉴를 올려 보시면 밑에 보이실 거예요. 그 결제창 혜택 위에 보시면은 그거 누르시면요, 지난달이랑 이번 달 실적이랑 인제 받은 혜택까지 확인이

In [24]:
import os
os.environ["RAG_GUIDANCE_DEBUG"] = "1"


In [25]:
res = await run_rag(query, session_state={})
res.get("debug")


[card_cache] hit=1 layer=redis key=02c53310b7363bd3
[rag] route=1.5ms retrieve=413.5ms cards=0.9ms post=0.1ms total=415.9ms docs=3 route=card_info cache=hit(redis) retrieve_cache=hit(redis)


{'used_policy_docs': [{'id': 'dadungi_013',
   'title': '서울시다둥이행복카드필수 생활비 적립 서비스 주말 배달앱 포인트 적립 - 배달앱 건당 2만원 이상 결제 시 1천 포인트 적립 편의점 포인트 적립',
   'source': 'service_guide_documents'},
  {'id': 'CARD-SHINHAN-신한카드-Point-Plan-(서울시다둥이행복카드)',
   'title': '테디카드 Point Plan (서울시다둥이행복카드)',
   'source': 'card_products'}],
 'used_consult_docs': [{'id': 'hana_consultation_23419',
   'title': '이용방법 안내 상담',
   'score': 0.4244357332923639},
  {'id': 'hana_consultation_27045',
   'title': '이벤트 안내 상담',
   'score': 0.4175280197684794}],
 'consult_hints': None,
 'consult_hints_enabled': True}

In [26]:
res = await run_rag(query, session_state={})
res.get("guidanceScript")


[card_cache] hit=1 layer=redis key=02c53310b7363bd3
[rag] route=1.6ms retrieve=494.1ms cards=1.0ms post=0.0ms total=496.8ms docs=3 route=card_info cache=hit(redis) retrieve_cache=hit(redis)


'Point Plan (서울시다둥이행복카드) 혜택부터 안내해 드릴까요'

In [27]:
res = await run_rag(query, session_state={})
res.get("guidanceScript")
res.get("debug", {}).get("consult_hints")


[card_cache] hit=1 layer=redis key=02c53310b7363bd3
[rag] route=1.6ms retrieve=353.8ms cards=1.2ms post=0.1ms total=356.6ms docs=3 route=card_info cache=hit(redis) retrieve_cache=hit(redis)


In [28]:
from app.rag.pipeline import route
from app.rag.tests.test_suite import TESTS

ids = {"T024","T028"}
for t in TESTS:
    if t["id"] not in ids:
        continue
    r = route(t["query"])
    print(t["id"], r.get("route"), r.get("filters") or r.get("boost"))


T024 card_info {}
T028 card_info {}


In [29]:
from app.rag.pipeline import route
from app.rag.tests.test_suite import TESTS

ids = {"T006","T007","T008","T009","T010","T024","T028","T031","T032","T034","T035"}
for t in TESTS:
    if t["id"] not in ids:
        continue
    routing = route(t["query"])
    print(t["id"], routing.get("route"), routing.get("db_route"), routing.get("filters") or routing.get("boost"))


T006 card_info both {'card_name': ['K-패스 테디카드 체크', 'K-패스 테디카드'], 'weak_intent': ['혜택'], 'benefit_type': ['다자녀']}
T007 card_info card_tbl {'card_name': ['K-패스 테디카드 체크', 'K-패스 테디카드'], 'benefit_type': ['다자녀']}
T008 card_info card_tbl {'card_name': ['K-패스 테디카드 체크', 'K-패스 테디카드'], 'region': ['경기']}
T009 card_info card_tbl {'card_name': ['K-패스 테디카드 체크', 'K-패스 테디카드'], 'region': ['충남']}
T010 card_info card_tbl {'card_name': ['K-패스 테디카드 체크'], 'benefit_type': ['체크']}
T024 card_info card_tbl {}
T028 card_info both {}
T031 card_info card_tbl {'card_name': ['KT 으랏차차 테디카드']}
T032 card_info card_tbl {'card_name': ['KT 으랏차차 테디카드']}
T034 card_usage guide_tbl {'intent': ['phone_lookup'], 'phone_lookup': True}
T035 card_usage guide_tbl {'intent': ['phone_lookup'], 'phone_lookup': True}


In [30]:
from app.rag.tests.test_suite import TESTS
from app.rag.pipeline import RAGConfig, run_rag

async def run_guidance_checks():
    for t in TESTS:
        res = await run_rag(t["query"], config=RAGConfig(top_k=4), session_state={})
        print(t["id"], t["query"])
        print(res.get("guidanceScript"))
        print("---")

await run_guidance_checks()


[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=90.7 rows=0
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=26.0 rows=8
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=127.6 rows=1
[retriever] source=guide_all fetch_ms=0.0 cand_added=8 total_cand=8 break_hit=False
[pipeline_retrieve] retrieve_ms=250.3 doc_count=6 route=card_usage
[cards] llm_input_chars=1030 ctx_chars=520 doc_ids=['narasarang_faq_011', 'narasarang_faq_005'] doc_chars=[237, 283]
[card_cache] set layer=redis key=cdf3947e0f28544a ttl=120
[card_cache] set layer=mem key=cdf3947e0f28544a ttl=120
[rag] route=1.4ms retrieve=540.7ms cards=1726.8ms post=0.2ms total=2269.0ms docs=6 route=card_usage cache=miss retrieve_cache=miss
T001 나라사랑 잃어버렸어요
나라사랑카드 분실 신고 및 재발급 절차를 안내드릴게요
---
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=144.9 rows=4
[retriever_db] table=service_guide_documents mode=text submode